In [13]:
import tensorflow as tf

import keras

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np
import os
import shutil
import sklearn
import cv2
import csv
import math


#import image
from skimage.io import imread
from PIL import Image
import matplotlib.image as mpimg
#导入图表库
import matplotlib.pyplot as plt
from keras import models
from keras import layers
from keras.layers import Dense
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from keras.utils import plot_model
# PCA 主成分分析
from sklearn.decomposition import PCA

from keras.callbacks import TensorBoard

print("hello tensorflow")

hello tensorflow


In [2]:
# os.listdir("../../emotionData/")

##  划分数据集

前 200000 训练数据集
后  50000 测试数据集

In [3]:
# N_train_samples = 200000
# train_data = csv_data.head(N_train_samples)
# print(train_data)

#df.index[np.where(np.isnan(df))[0]]

# train_data.to_csv("../../emotionData/data/train_data_write.csv",index=None)

In [4]:
X_train = pd.read_csv(open("../../emotionData/data/train/train_data.csv",encoding='utf-8'))
y_train = pd.read_csv(open("../../emotionData/data/train/train_label.csv",encoding='utf-8'))

print("train data shape:   ",X_train.shape)
print("train label shape:  ",y_train.shape)

train data shape:    (199995, 128)
train label shape:   (199995, 1)


In [5]:
# test = pd.read_csv(open("../../emotionData/data/test/test_data_.csv",encoding='utf-8'))
# N_test_samples = 50000
# test_data = test.tail(N_test_samples)
# print(test_data)
# test_data.to_csv("../../emotionData/data/test/test_data.csv",index=None)

In [6]:
X_test = pd.read_csv(open("../../emotionData/data/test/test_data.csv",encoding='utf-8'))
y_test = pd.read_csv(open("../../emotionData/data/test/test_label.csv",encoding='utf-8'))


print("test data shape:   ",X_test.shape)
print("test label shape:  ",y_test.shape)

test data shape:    (49999, 128)
test label shape:   (49999, 1)


## 准备数据

对数据进行归一化 (0 - 1)

In [7]:
X_train_normolized = X_train
#最小-最大规范化
X_train_normolized = (X_train_normolized - X_train_normolized.min())/(X_train_normolized.max() - X_train_normolized.min()) 
# X_train_normolized.to_csv("../../emotionData/data/train/X_train_normolized.csv",index=None)
X_train_normolized = X_train_normolized.values



X_test_normolized = X_test
#最小-最大规范化
X_test_normolized = (X_test_normolized - X_test_normolized.min())/(X_test_normolized.max() - X_test_normolized.min()) 
# X_test_normolized.to_csv("../../emotionData/data/test/X_test_normolized.csv",index=None)
X_test_normolized = X_test_normolized.values




In [20]:
model = Sequential()

model.add(Dense(128, input_dim=128, activation='relu'))

model.add(Dense(64, activation='tanh'))

model.add(Dense(64, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

model.compile(loss='mean_squared_error', optimizer='adam')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               16512     
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 64)                4160      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 65        
Total params: 28,993
Trainable params: 28,993
Non-trainable params: 0
_________________________________________________________________


In [21]:
tbCallBack = TensorBoard(
                 log_dir='./logs',  # log 目录
                 histogram_freq=0,  # 按照何等频率（epoch）来计算直方图，0为不计算
#                  batch_size=32,     # 用多大量的数据计算直方图
                 write_graph=True,  # 是否存储网络结构图
                 write_grads=True, # 是否可视化梯度直方图
                 write_images=True,# 是否可视化参数
                 embeddings_freq=0,
                 embeddings_layer_names=None, 
                 embeddings_metadata=None)


## 定义模型

    搭建了一个单隐层的神经网络。
    因为是回归问题，在输出层作者并没有使用激活函数。
    然后使用Scikit-Learn接口包装器将Sequential模型作为Scikit-Learn工作流的一部分。
    最后使用了交叉验证方法来评估模型好坏。 

In [32]:
his=model.fit(
    X_train_normolized,
    y_train,
    
    validation_data=(X_test_normolized,y_test),
    epochs=500,
    batch_size=128,
    
    shuffle=False,
    verbose=1,
    callbacks=[tbCallBack]
)

Train on 199995 samples, validate on 49999 samples
Epoch 1/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0077 - val_loss: 0.0671
Epoch 2/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0076 - val_loss: 0.0663
Epoch 3/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0078 - val_loss: 0.0680
Epoch 4/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0077 - val_loss: 0.0663
Epoch 5/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0075 - val_loss: 0.0687
Epoch 6/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0075 - val_loss: 0.0680
Epoch 7/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0074 - val_loss: 0.0676
Epoch 8/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0074 - val_loss: 0.0676
Epoch 9/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0

Epoch 76/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0068 - val_loss: 0.0723
Epoch 77/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0067 - val_loss: 0.0723
Epoch 78/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0067 - val_loss: 0.0718
Epoch 79/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0067 - val_loss: 0.0722
Epoch 80/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0067 - val_loss: 0.0726
Epoch 81/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0067 - val_loss: 0.0728
Epoch 82/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0070 - val_loss: 0.0733
Epoch 83/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0068 - val_loss: 0.0733
Epoch 84/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0068 - val_loss: 0.0733
Epoch 85/500
199995

199995/199995 [==============================] - 2s 11us/step - loss: 0.0063 - val_loss: 0.0730
Epoch 151/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0741
Epoch 152/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0731
Epoch 153/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0737
Epoch 154/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0727
Epoch 155/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0735
Epoch 156/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0062 - val_loss: 0.0709
Epoch 157/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0061 - val_loss: 0.0734
Epoch 158/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0060 - val_loss: 0.0729
Epoch 159/500
199995/199

199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0734
Epoch 225/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0718
Epoch 226/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0060 - val_loss: 0.0714
Epoch 227/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0059 - val_loss: 0.0728
Epoch 228/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0059 - val_loss: 0.0725
Epoch 229/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0727
Epoch 230/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0059 - val_loss: 0.0725
Epoch 231/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0731
Epoch 232/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0059 - val_loss: 0.0732
Epoch 233/500
199995/199

199995/199995 [==============================] - 2s 11us/step - loss: 0.0057 - val_loss: 0.0713
Epoch 299/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0056 - val_loss: 0.0705
Epoch 300/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0713
Epoch 301/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0058 - val_loss: 0.0723
Epoch 302/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0056 - val_loss: 0.0724
Epoch 303/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0056 - val_loss: 0.0713
Epoch 304/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0056 - val_loss: 0.0716
Epoch 305/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0057 - val_loss: 0.0698
Epoch 306/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0057 - val_loss: 0.0709
Epoch 307/500
199995/199

199995/199995 [==============================] - 2s 11us/step - loss: 0.0055 - val_loss: 0.0744
Epoch 373/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0745
Epoch 374/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0741
Epoch 375/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0741
Epoch 376/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0745
Epoch 377/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0053 - val_loss: 0.0750
Epoch 378/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0746
Epoch 379/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0734
Epoch 380/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0743
Epoch 381/500
199995/199

199995/199995 [==============================] - 2s 11us/step - loss: 0.0053 - val_loss: 0.0741
Epoch 447/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0053 - val_loss: 0.0746
Epoch 448/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0752
Epoch 449/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0746
Epoch 450/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0053 - val_loss: 0.0747
Epoch 451/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0738
Epoch 452/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0736
Epoch 453/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0054 - val_loss: 0.0741
Epoch 454/500
199995/199995 [==============================] - 2s 11us/step - loss: 0.0053 - val_loss: 0.0760
Epoch 455/500
199995/199

In [30]:
# 做预测

ynew = model.predict_proba(X_test_normolized)

# 显示输入输出

#for i in range(len(X_test_normolized)):
#    print("X=%s, Predicted=%s" % (i, ynew[i]))

In [31]:

result = (y_test.values - ynew)

#for i in range(len(X_test_normolized)):
#    print(result[i])
#print(ynew.dtypes)

count = 0
per = 0

for c in result:
    if(-0.1 <= c and c <= 0.1):
        count = count + 1

per = count / len(result)

print(per)

#c = sum(result)/len(result)
#c = math.sqrt(c)
#print(c)

0.3562671253425069


In [38]:
predicts = model.predict(X_test_normolized)
print(predicts)
print(y_test)

[[0.49429822]
 [0.5930854 ]
 [0.5979461 ]
 ...
 [0.77103853]
 [0.63325167]
 [0.56466293]]
       0.395123
0      0.964142
1      0.822079
2      0.804279
3      0.800226
4      0.381252
5      0.765708
6      0.787427
7      0.769957
8      0.902932
9      0.843410
10     0.375690
11     0.915383
12     0.957649
13     0.968612
14     0.908518
15     0.690643
16     0.970275
17     0.878559
18     0.910977
19     0.605759
20     0.262652
21     0.597588
22     0.307492
23     0.108907
24     0.072813
25     0.889047
26     0.198438
27     0.924631
28     0.971345
29     0.307860
...         ...
49969  0.568878
49970  0.633470
49971  0.780266
49972  0.633978
49973  0.893746
49974  0.868698
49975  0.855433
49976  0.843027
49977  0.451989
49978  0.508939
49979  0.655955
49980  0.752893
49981  0.344012
49982  0.298667
49983  0.282969
49984  0.334649
49985  0.135420
49986  0.250442
49987  0.733053
49988  0.129027
49989  0.813076
49990  0.713258
49991  0.904694
49992  0.865130
49993  0.81880

In [14]:
# define base model
# define base mode
#def baseline_model():
    # create model
model = Sequential()
model.add(Dense(128, input_dim=128, init='normal', activation='relu')) 
model.add(Dense(64, input_dim=128, init='normal', activation='relu'))
model.add(Dense(1, init='normal', activation='relu'))
      # Compile model
model.compile(loss='mean_squared_error', optimizer='adam', metrics=["accuracy"]) 
 #   return model


D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(128, input_dim=128, activation="relu", kernel_initializer="normal")`
  
D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(64, input_dim=128, activation="relu", kernel_initializer="normal")`
  import sys
D:\anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="relu", kernel_initializer="normal")`
  


我们将随机数生成器初始化为常量值（7）
这对于确保我们可以再次精确地实现从该模型获得的结果非常重要，它确保可以再现训练神经网络模型的随机过程。

In [9]:
# fix random seed for reproducibility
seed = 7
np.random.seed(seed)

In [10]:
# evaluate model with standardized dataset
estimator = KerasRegressor(
    build_fn=baseline_model, 
    nb_epoch=100,
    # batch_size
    # 通常设为2的n次幂，常用的包括64,128,256                      
    # 网络较小时选用256，较大时选用64
    batch_size=256,
    verbose=0
)

NameError: name 'baseline_model' is not defined

## 训练

In [11]:
#model.fit(x_train, y_train, nb_epoch=64, batch_size=20, verbose=0)   
#model.fit(X_train_normolized, y_train, batch_size=32, epochs=100, shuffle=True,verbose=0,validation_split=0.2)
#print(hist.history)
#score = model.evaluate(X_test_normolized, y_test, batch_size=10)

In [12]:
# Fit the model
model.fit(
    X_train_normolized, 
    y_train,
    batch_size=32, 
    epochs=10, 
    verbose=1, 
    callbacks=None,
    validation_split=0.3, 
    validation_data=None, 
    shuffle=True, 
    class_weight=None, 
    sample_weight=None, 
    initial_epoch=0
)

Instructions for updating:
Use tf.cast instead.
Train on 140000 samples, validate on 60000 samples
Epoch 1/10
140000/140000 [==============================] - 4s 28us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 2/10
140000/140000 [==============================] - 4s 27us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 3/10
140000/140000 [==============================] - 4s 27us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 4/10
140000/140000 [==============================] - 4s 27us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 5/10
140000/140000 [==============================] - 4s 26us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 6/10
140000/140000 [==============================] - 4s 26us/step - loss: nan - acc: 0.0000e+00 - val_loss: nan - val_acc: 0.0000e+00
Epoch 7/10
140000/140000 [=============================

### 交叉验证
使用10倍交叉验证程序（kfold）在我们的数据集（X和dummy_y）上评估我们的模型（估计器）